Добавление шрифтов

In [ ]:
import os

# Создаем папку для шрифтов
os.makedirs("/content/fonts", exist_ok=True)

# Перемещаем все шрифты в эту папку
for font_file in os.listdir("/content/"):
    if font_file.lower().endswith(".ttf"):
        os.rename(f"/content/{font_file}", f"/content/fonts/{font_file}")

print("Все шрифты перемещены в папку /content/fonts")

Добавление следов (кофе, крови, клякс и следов от пальцев)

In [ ]:
import os
from google.colab import files

def create_stain_folder(folder_path="/content/stains"):
    os.makedirs(folder_path, exist_ok=True)
    print(f" Папка создана: {folder_path}")
    return folder_path

def upload_stain_images(destination="/content/stains"):
    print("Загрузите PNG-файлы пятен (например, coffee1.png, blood1.png):")
    uploaded = files.upload()
    for filename in uploaded.keys():
        os.rename(filename, os.path.join(destination, filename))
    print("Файлы успешно добавлены в:", destination)

# Использование:
STAINS_FOLDER = create_stain_folder()
upload_stain_images(STAINS_FOLDER)

Генератор листов A4 (предварительно загружены шрифты, загруженные в папку fonts, загружен txt с армянским текстом и загружены png-фото следов)

In [ ]:
import os
import random
import re
import json
import numpy as np
from PIL import Image, ImageDraw, ImageFont, ImageEnhance, ImageFilter
from tqdm import tqdm

# Константы
TEXT_FILE_PATH = "/content/Tumanian.txt"
FONTS_FOLDER = "/content/fonts"
STAINS_FOLDER = "/content/stains"
OUTPUT_FOLDER = "/content/generated_pages"
A4_SIZE_PX = (1240, 1754)  # Ширина × Высота
MARGIN = 150
FONT_SIZE = 42
LINE_SPACING = 14
PAGES_TO_GENERATE = 2000
PIVOT_MARGIN = 100

def check_files():
    """Проверка наличия файлов."""
    if not os.path.exists(TEXT_FILE_PATH):
        raise FileNotFoundError(f"Файл с текстом не найден: {TEXT_FILE_PATH}")
    print(f"Файл с текстом найден: {TEXT_FILE_PATH}")
    os.makedirs(FONTS_FOLDER, exist_ok=True)
    global font_files
    font_files = [os.path.join(FONTS_FOLDER, f) for f in os.listdir(FONTS_FOLDER) if f.lower().endswith(".ttf")]
    if not font_files:
        raise FileNotFoundError(f"Шрифты не найдены в папке {FONTS_FOLDER}")
    print(f"Найдено шрифтов: {len(font_files)}")
    os.makedirs(STAINS_FOLDER, exist_ok=True)
    stain_files = [os.path.join(STAINS_FOLDER, f) for f in os.listdir(STAINS_FOLDER) if f.endswith(".png")]
    print(f"Найдено PNG-файлов пятен: {len(stain_files)}" if stain_files else "Пятна не найдены.")

def load_text():
    """Загрузка и фильтрация текста."""
    with open(TEXT_FILE_PATH, "r", encoding="utf-8") as f:
        raw_text = f.read()
    filtered_text = re.sub(r"[^Ա-Ֆա-ֆՙ-՟։՞,\s]", "", raw_text)
    words = re.sub(r'\s+', ' ', filtered_text).strip().split()
    return words

def load_fonts():
    """Загрузка шрифтов."""
    return font_files

def create_aged_background(size):
    """Создание фона с эффектом старения."""
    paper_colors = [
        (255, 255, 255),  # Белый
        (250, 245, 230),  # Кремовый
        (245, 240, 220),  # Светло-бежевый
        (245, 235, 190),  # Светло-жёлтый
        (240, 235, 210),  # Серо-жёлтый
        (230, 220, 190),  # Пергаментный
        (220, 215, 195),  # Слегка коричневатый
    ]
    base_color = random.choice(paper_colors)
    base = Image.new("RGB", size, base_color)
    draw = ImageDraw.Draw(base)
    for _ in range(2000):
        x, y = random.randint(0, size[0]-1), random.randint(0, size[1]-1)
        shade = random.randint(170, 255)
        draw.point((x, y), fill=(shade, shade, shade))
    return base

def draw_text(base_img, words, font):
    """Отрисовка текста."""
    draw = ImageDraw.Draw(base_img)
    word_positions = []
    line_positions = []
    max_width = base_img.width - 2 * MARGIN
    y = MARGIN
    while y + FONT_SIZE < base_img.height - MARGIN:
        line = ""
        x = MARGIN
        line_words = []
        while True:
            word = random.choice(words)
            test_line = (line + " " + word).strip()
            if draw.textlength(test_line, font=font) > max_width:
                break
            line = test_line
            line_words.append(word)
        x_cursor = x
        line_start_y = y
        for word in line.split():
            color = (0, 0, 0)
            word_img = Image.new("RGBA", base_img.size, (0, 0, 0, 0))
            word_draw = ImageDraw.Draw(word_img)
            word_draw.text((x_cursor, y), word, font=font, fill=color)
            bbox = word_img.getbbox()
            if bbox:
                x1, y1, x2, y2 = bbox
                if x2 - x1 >= 5 and y2 - y1 >= 5:
                    word_crop = word_img.crop((x1, y1, x2, y2))
                    base_img.paste(word_crop, (x1, y1), word_crop)
                    word_positions.append((x1, y1, x2, y2, word))
            x_cursor += int(draw.textlength(word + " ", font=font))
        if line_words:
            line_positions.append((line_start_y, line_words))
        y += FONT_SIZE + LINE_SPACING
    return base_img, word_positions, line_positions

def apply_effects(img, word_positions, line_positions):
    """Применение эффектов без поворота."""
    img = apply_simple_blur(img, word_positions)
    img = apply_simple_bad_print(img, line_positions)
    if random.random() < 0.8:
        img = add_scanner_noise(img)
    img = add_printer_jam_stripes(img)
    img = add_tear_edges(img)
    img = add_random_stains(img)
    img = adjust_brightness_contrast(img)
    return img

def apply_simple_blur(img, word_positions, probability=0.4):
    """Добавление размытия."""
    if random.random() > probability or not word_positions:
        return img
    img = img.convert("RGBA")
    selected_words = random.sample(word_positions, k=min(3, len(word_positions)))
    for x1, y1, x2, y2, _ in selected_words:
        padding = 5
        x1_padded = max(x1 - padding, 0)
        y1_padded = max(y1 - padding, 0)
        x2_padded = min(x2 + padding, img.width)
        y2_padded = min(y2 + padding, img.height)
        region = img.crop((x1_padded, y1_padded, x2_padded, y2_padded))
        if region.width < 5 or region.height < 5:
            continue
        blurred = region.filter(ImageFilter.GaussianBlur(radius=random.uniform(1.0, 2.0)))
        img.paste(blurred, (x1_padded, y1_padded), blurred.split()[3])
    return img.convert("RGB")

def apply_simple_bad_print(img, line_positions, probability=0.4):
    """Эффект плохой печати."""
    if random.random() > probability or not line_positions:
        return img
    img = img.convert("RGBA")
    mask = Image.new("RGBA", img.size, (0, 0, 0, 0))
    draw = ImageDraw.Draw(mask)
    if random.random() < 0.25:
        selected_lines = random.sample(line_positions, k=min(2, len(line_positions)))
        for line_y, words in selected_lines:
            num_words_to_erase = int(len(words) * random.uniform(0.5, 0.75))
            if num_words_to_erase == 0:
                continue
            words_to_erase = random.sample(words, k=num_words_to_erase)
            x_cursor = MARGIN
            for word in words:
                if word in words_to_erase:
                    word_img = Image.new("RGBA", img.size, (0, 0, 0, 0))
                    word_draw = ImageDraw.Draw(word_img)
                    word_draw.text((x_cursor, line_y), word, font=ImageFont.truetype(font_files[0], FONT_SIZE), fill=(0, 0, 0))
                    bbox = word_img.getbbox()
                    if bbox:
                        x1, y1, x2, y2 = bbox
                        if x2 - x1 >= 5 and y2 - y1 >= 5:
                            color = (250, 245, 230, 220)
                            draw.rectangle((x1, y1, x2, y2), fill=color)
                x_cursor += int(draw.textlength(word + " ", font=ImageFont.truetype(font_files[0], FONT_SIZE)))
    else:
        selected_lines = random.sample(line_positions, k=min(2, len(line_positions)))
        for line_y, words in selected_lines:
            x_cursor = MARGIN
            for word in words:
                word_img = Image.new("RGBA", img.size, (0, 0, 0, 0))
                word_draw = ImageDraw.Draw(word_img)
                word_draw.text((x_cursor, line_y), word, font=ImageFont.truetype(font_files[0], FONT_SIZE), fill=(0, 0, 0))
                bbox = word_img.getbbox()
                if bbox:
                    x1, y1, x2, y2 = bbox
                    if x2 - x1 >= 5 and y2 - y1 >= 5:
                        region = img.crop((x1, y1, x2, y2))
                        region_array = np.array(region)
                        gray = np.mean(region_array[:, :, :3], axis=2) if region_array.shape[2] == 4 else region_array
                        text_mask = gray < 100
                        speckle_mask = (np.random.rand(*gray.shape) < 0.4) & text_mask
                        region_array[:, :, :3][speckle_mask] = [250, 245, 230]
                        region_array[:, :, 3][speckle_mask] = 255
                        region = Image.fromarray(region_array)
                        img.paste(region, (x1, y1), region.split()[3])
                x_cursor += int(draw.textlength(word + " ", font=ImageFont.truetype(font_files[0], FONT_SIZE)))
    img = Image.alpha_composite(img, mask)
    return img.convert("RGB")

def add_scanner_noise(img, noise_std=8):
    """Шум сканирования."""
    img_array = np.array(img)
    noise = np.random.normal(0, noise_std, img_array.shape).astype(np.float32)
    img_array = img_array.astype(np.float32) + noise
    img_array = np.clip(img_array, 0, 255).astype(np.uint8)
    return Image.fromarray(img_array)

def add_printer_jam_stripes(img, probability=0.3):
    """Полосы замятия."""
    if random.random() > probability:
        return img
    img = img.convert("RGBA")
    mask = Image.new("RGBA", img.size, (0, 0, 0, 0))
    draw = ImageDraw.Draw(mask)
    for _ in range(random.randint(1, 2)):
        y = random.randint(100, img.height - 100)
        height = random.randint(20, 50)
        shadow_color = (150, 150, 150, 60)
        for i in range(height):
            alpha = int(30 * (1 - abs((i - height / 2) / (height / 2))))
            draw.line((0, y + i, img.width, y + i), fill=(200, 200, 200, alpha))
        draw.line((0, y, img.width, y), fill=shadow_color, width=2)
        draw.line((0, y + height, img.width, y + height), fill=shadow_color, width=2)
    img_array = np.array(img)
    for stripe_y in range(y, y + height):
        if stripe_y >= img.height:
            break
        shift = random.randint(-2, 2)
        img_array[stripe_y] = np.roll(img_array[stripe_y], shift, axis=0)
    img = Image.fromarray(img_array)
    img = Image.alpha_composite(img, mask)
    return img.convert("RGB")

def add_tear_edges(img, probability=0.2):
    """Разрывы по краям."""
    if random.random() > probability:
        return img
    draw = ImageDraw.Draw(img)
    edges = random.sample(["left", "right", "top", "bottom"], k=random.randint(1, 2))
    for edge in edges:
        if edge == "left":
            for _ in range(random.randint(5, 10)):
                x = random.randint(0, 20)
                y = random.randint(0, img.height)
                length = random.randint(10, 30)
                points = [(x, y), (x + random.randint(5, 15), y + random.randint(-5, 5)),
                          (x + random.randint(5, 15), y + length), (x, y + length)]
                draw.polygon(points, fill=(245, 240, 220), outline=(200, 195, 190))
        elif edge == "right":
            for _ in range(random.randint(5, 10)):
                x = random.randint(img.width - 20, img.width)
                y = random.randint(0, img.height)
                length = random.randint(10, 30)
                points = [(x, y), (x - random.randint(5, 15), y + random.randint(-5, 5)),
                          (x - random.randint(5, 15), y + length), (x, y + length)]
                draw.polygon(points, fill=(245, 240, 220), outline=(200, 195, 190))
        elif edge == "top":
            for _ in range(random.randint(5, 10)):
                x = random.randint(0, img.width)
                y = random.randint(0, 20)
                length = random.randint(10, 30)
                points = [(x, y), (x + random.randint(-5, 5), y + random.randint(5, 15)),
                          (x + length, y + random.randint(5, 15)), (x + length, y)]
                draw.polygon(points, fill=(245, 240, 220), outline=(200, 195, 190))
        elif edge == "bottom":
            for _ in range(random.randint(5, 10)):
                x = random.randint(0, img.width)
                y = random.randint(img.height - 20, img.height)
                length = random.randint(10, 30)
                points = [(x, y), (x + random.randint(-5, 5), y - random.randint(5, 15)),
                          (x + length, y - random.randint(5, 15)), (x + length, y)]
                draw.polygon(points, fill=(245, 240, 220), outline=(200, 195, 190))
    return img

def add_random_stains(base_img, probability=0.4):
    """Случайные пятна."""
    if random.random() > probability:
        return base_img
    stain_files = [os.path.join(STAINS_FOLDER, f) for f in os.listdir(STAINS_FOLDER) if f.endswith(".png")]
    if not stain_files:
        return base_img
    stain_path = random.choice(stain_files)
    stain = Image.open(stain_path).convert("RGBA")
    if "coffee" in stain_path.lower():
        scale = random.uniform(0.7, 0.9)
    elif "blot" in stain_path.lower():
        scale = random.uniform(0.05, 0.15)
    elif "finger" in stain_path.lower():
        scale = random.uniform(0.1, 0.2)
    else:
        scale = random.uniform(0.3, 0.5)
    max_scale = min(A4_SIZE_PX[0] / stain.width, A4_SIZE_PX[1] / stain.height)
    scale = min(scale, max_scale * 0.95)
    stain = stain.resize((int(stain.width * scale), int(stain.height * scale)), Image.LANCZOS)
    stain = stain.rotate(random.randint(0, 360), expand=True)
    alpha = random.randint(180, 255)
    r, g, b, a = stain.split()
    a = a.point(lambda p: min(p * (alpha / 255), 255))
    stain.putalpha(a)
    max_x = max(0, base_img.width - stain.width)
    max_y = max(0, base_img.height - stain.height)
    if max_x < 0 or max_y < 0:
        return base_img
    pos = (random.randint(0, max_x), random.randint(0, max_y))
    base_img = base_img.convert("RGBA")
    base_img.paste(stain, pos, stain)
    return base_img.convert("RGB")

def adjust_brightness_contrast(img, probability=0.7):
    """Регулировка яркости и контраста."""
    if random.random() > probability:
        return img
    brightness_factor = random.uniform(1.0, 1.1)
    contrast_factor = random.uniform(0.95, 1.05)
    img = ImageEnhance.Brightness(img).enhance(brightness_factor)
    img = ImageEnhance.Contrast(img).enhance(contrast_factor)
    return img

def enforce_a4_size(img, word_positions):
    """Приведение к вертикальному A4 без дополнительного поворота."""
    width, height = img.size
    target_width, target_height = A4_SIZE_PX

    # Создаём новое изображение с фиксированным размером A4
    new_img = Image.new("RGB", A4_SIZE_PX, (250, 245, 230))
    new_positions = word_positions

    # Если изображение горизонтальное, мы его не поворачиваем, а корректируем координаты
    if width > height:
        # Масштабируем изображение
        scale = min(target_width / height, target_height / width)  # Меняем местами width и height
        new_width = int(height * scale)  # Теперь height становится шириной
        new_height = int(width * scale)   # А width — высотой
        img = img.rotate(90, expand=True, fillcolor=(250, 245, 230))
        img = img.resize((new_width, new_height), Image.LANCZOS)

        # Корректируем координаты (с учётом поворота на 90°)
        new_positions = [(height - y2, x1, height - y1, x2, word) for x1, y1, x2, y2, word in word_positions]
        new_positions = [(int(x1 * scale), int(y1 * scale), int(x2 * scale), int(y2 * scale), word)
                         for x1, y1, x2, y2, word in new_positions]
    else:
        # Масштабируем без поворота
        scale = min(target_width / width, target_height / height)
        new_width = int(width * scale)
        new_height = int(height * scale)
        img = img.resize((new_width, new_height), Image.LANCZOS)
        new_positions = [(int(x1 * scale), int(y1 * scale), int(x2 * scale), int(y2 * scale), word)
                         for x1, y1, x2, y2, word in word_positions]

    # Центрируем изображение на новом холсте A4
    offset_x = (target_width - new_width) // 2
    offset_y = (target_height - new_height) // 2
    new_img.paste(img, (offset_x, offset_y))
    new_positions = [(x1 + offset_x, y1 + offset_y, x2 + offset_x, y2 + offset_y, word)
                     for x1, y1, x2, y2, word in new_positions]

    return new_img, new_positions

def save_annotations(word_positions, image_name, output_folder, page_idx):
    """Сохранение аннотаций в формате Label Studio."""
    width_image, height_image = A4_SIZE_PX
    annotations = []
    for idx, (x1, y1, x2, y2, word) in enumerate(word_positions):
        # Преобразуем координаты в проценты
        x_percent = (x1 / width_image) * 100
        y_percent = (y1 / height_image) * 100
        width_percent = ((x2 - x1) / width_image) * 100
        height_percent = ((y2 - y1) / height_image) * 100

        # Уникальный идентификатор для слова
        word_id = f"word_{page_idx}_{idx}"

        # Аннотация для bounding box
        bbox_annotation = {
            "id": word_id,
            "type": "rectanglelabels",
            "value": {
                "x": round(x_percent, 2),
                "y": round(y_percent, 2),
                "width": round(width_percent, 2),
                "height": round(height_percent, 2),
                "rectanglelabels": ["Text"]
            },
            "from_name": "label",
            "to_name": "image"
        }

        # Аннотация для текста
        text_annotation = {
            "id": word_id,
            "type": "textarea",
            "value": {
                "text": [word]
            },
            "from_name": "transcription",
            "to_name": "image"
        }

        annotations.append(bbox_annotation)
        annotations.append(text_annotation)

    # Формируем JSON для Label Studio
    task = {
        "data": {
            "image": os.path.join(OUTPUT_FOLDER, image_name)
        },
        "predictions": [
            {
                "result": annotations
            }
        ]
    }

    # Сохраняем JSON для каждой страницы
    annotation_path = os.path.join(output_folder, f"{os.path.splitext(image_name)[0]}_labelstudio.json")
    with open(annotation_path, 'w', encoding='utf-8') as f:
        json.dump(task, f, ensure_ascii=False, indent=4)

def generate_page(font_path):
    """Генерация страницы."""
    font = ImageFont.truetype(font_path, FONT_SIZE)
    base_img = create_aged_background(A4_SIZE_PX)
    base_img, word_positions, line_positions = draw_text(base_img, words, font)
    base_img = apply_effects(base_img, word_positions, line_positions)

    # Принудительное приведение к A4
    base_img, word_positions = enforce_a4_size(base_img, word_positions)

    return base_img, word_positions

def generate_documents(font_path, count=2000):
    """Генерация документов."""
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    for page_idx in tqdm(range(count), desc="Generating pages"):
        while True:
            base_img, word_positions = generate_page(font_path)
            width, height = base_img.size
            if width <= height:  # Только вертикальные изображения
                # Пересоздаём изображение для сброса метаданных
                final_img = Image.new("RGB", (width, height), (250, 245, 230))
                final_img.paste(base_img, (0, 0))
                output_path = os.path.join(OUTPUT_FOLDER, f"page_{page_idx:03}.jpg")
                final_img.save(output_path, quality=95)
                # Сохраняем аннотации для Label Studio
                save_annotations(word_positions, f"page_{page_idx:03}.jpg", OUTPUT_FOLDER, page_idx)
                print(f"Сохранено: {output_path}")
                break
            print(f"Изображение горизонтальное, перегенерация для page_{page_idx:03}...")

if __name__ == "__main__":
    check_files()
    words = load_text()
    font_files = load_fonts()
    if font_files:
        font_path = font_files[0]
        generate_documents(font_path, count=PAGES_TO_GENERATE)
    else:
        print(f"Ошибка: шрифты не найдены в {FONTS_FOLDER}.")